In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import for the CRUD module 
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Username and password for database
username = "aacuser"
password = "December"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# Exclude the MongoDB internal '_id' field from queries
MONGO_PROJECTION = {"_id": 0}

# Centralized filter definitions instead of long if/elif chains
FILTERS = {
    "all": {},
    "water": {
        "breed": {
            "$in": [
                "Labrador Retriever",
                "Great Dane",
                "Newfoundland",
                "Beagle Mix",
            ]
        }
    },
    "wilderness": {
        "breed": {
            "$in": [
                "German Shepherd",
                "Alaskan Malamute",
                "Siberian Husky",
            ]
        }
    },
    "tracking": {
        "breed": {
            "$in": [
                "Bloodhound",
                "Bullmastiff",
                "American Bulldog Mix",
                "Maltese Mix",
            ]
        }
    },
    "disaster": {
        "breed": {
            "$in": [
                "Golden Retriever",
                "Cairn Terrier Mix",
                "Pit Bull Mix",
            ]
        }
    },
}

# Load initial data once for default view
df = pd.DataFrame.from_records(db.read({}, MONGO_PROJECTION))

# MongoDB v5+ is going to return the '_id' column if not projected out.
# We already project it out, but this is a safety net.
df.drop(columns=["_id"], inplace=True, errors="ignore")

# Define column names for clarity and safety
BREED_COL = "breed"
NAME_COL = "name"
LAT_COL = "location_latitude"
LON_COL = "location_longitude"

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add in Grazioso Salvare’s logo
image_filename = "Grazioso_Salvare_Logo.png"
encoded_image = base64.b64encode(open(image_filename, "rb").read())

# Base style for zebra striping in table rows
BASE_STYLE_DATA_CONDITIONAL = [
    {
        "if": {"row_index": "odd"},
        "backgroundColor": "rgb(248, 248, 248)",
    }
]

# Build the layout for the app
app.layout = html.Div(
    [
        # The header with title
        html.Div(
            [
                html.Img(
                    src="data:image/png;base64,{}".format(encoded_image.decode()),
                    style={"height": "100px", "float": "left"},
                ),
                html.H1(
                    "Grazioso Salvare Dashboard - Christopher Diaz",
                    style={"text-align": "center", "margin-left": "20px"},
                ),
            ],
            style={"display": "flex", "align-items": "center", "justify-content": "center"},
        ),
        html.Hr(),
        # Filter controls
        html.Div(
            [
                html.H3("Filter by Rescue Type: "),
                dcc.RadioItems(
                    id="filter-type",
                    options=[
                        {"label": "All Animals / Reset", "value": "all"},
                        {"label": "Water Rescue", "value": "water"},
                        {"label": "Mountain / Wilderness Rescue", "value": "wilderness"},
                        {"label": "Individual Tracking", "value": "tracking"},
                        {"label": "Natural Disaster", "value": "disaster"},
                    ],
                    value="all",
                    labelStyle={"display": "block", "margin-bottom": "10px"},
                ),
            ],
            style={"margin-bottom": "20px"},
        ),
        html.Hr(),
        # Interactive table
        dash_table.DataTable(
            id="datatable-id",
            columns=[
                {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
            ],
            data=df.to_dict("records"),
            page_size=10,
            sort_action="native",
            sort_mode="multi",
            filter_action="native",
            row_selectable="single",
            selected_rows=[0],
            style_cell={"textAlign": "left", "padding": "5px"},
            style_header={
                "backgroundColor": "rgb(230, 230, 230)",
                "fontWeight": "bold",
            },
            style_table={"overflowX": "auto"},
            style_data_conditional=BASE_STYLE_DATA_CONDITIONAL,
        ),
        html.Br(),
        html.Hr(),
        # Side by side chart layout
        html.Div(
            className="row",
            style={"display": "flex"},
            children=[
                html.Div(
                    id="graph-id",
                    className="col s12 m6",
                    style={"width": "50%", "padding": "10px"},
                ),
                html.Div(
                    id="map-id",
                    className="col s12 m6",
                    style={"width": "50%", "padding": "10px"},
                ),
            ],
        ),
    ]
)

#############################################
# Interaction Between Components / Controller
#############################################

# Update the data table based on filter selection
@app.callback(Output("datatable-id", "data"), [Input("filter-type", "value")])
def update_dashboard(filter_type):
    """
    Update the table data in response to radio button selection.
    Uses a centralized FILTERS dictionary to avoid long if/elif chains.
    """
    query = FILTERS.get(filter_type, {})
    dff = pd.DataFrame.from_records(db.read(query, MONGO_PROJECTION))
    dff.drop(columns=["_id"], inplace=True, errors="ignore")
    return dff.to_dict("records")


# Display the breeds of animals based on quantity represented in the data table
@app.callback(
    Output("graph-id", "children"),
    [Input("datatable-id", "derived_virtual_data")],
)
def update_graphs(viewData):
    """
    Build a pie chart of breeds based on the current table view.
    Uses group-by/value_counts for efficient aggregation.
    """
    if viewData is None or len(viewData) == 0:
        dff = df.copy()
    else:
        dff = pd.DataFrame(viewData)

    if BREED_COL not in dff.columns:
        return html.Div("Breed column missing")

    # Count occurrences of each breed for the chart
    breed_counts = dff[BREED_COL].value_counts().reset_index()
    breed_counts.columns = [BREED_COL, "count"]

    fig = px.pie(
        breed_counts,
        names=BREED_COL,
        values="count",
        title="Breed Distribution (Current Filter)",
    )

    return [dcc.Graph(figure=fig)]


# Highlight selected columns in data table
@app.callback(
    Output("datatable-id", "style_data_conditional"),
    [Input("datatable-id", "selected_columns")],
)
def update_styles(selected_columns):
    """
    Highlight selected columns while keeping base zebra striping.
    """
    if not selected_columns:
        return BASE_STYLE_DATA_CONDITIONAL

    highlight_styles = [
        {
            "if": {"column_id": col},
            "background_color": "#D2F3FF",
        }
        for col in selected_columns
    ]

    # Combine base striping with highlighted columns
    return BASE_STYLE_DATA_CONDITIONAL + highlight_styles


# Update to geographic map based on selected row in data table
@app.callback(
    Output("map-id", "children"),
    [
        Input("datatable-id", "derived_virtual_data"),
        Input("datatable-id", "derived_virtual_selected_rows"),
    ],
)
def update_map(viewData, index):
    """
    Update the map based on the selected row in the table.
    Uses named columns instead of hard-coded indices for safety and clarity.
    """
    if viewData is None or len(viewData) == 0:
        return

    dff = pd.DataFrame.from_dict(viewData)

    # Default to first row if no selection
    if not index:
        row = 0
    else:
        row = index[0]

    # Guard in case row index goes out of range
    if row >= len(dff):
        row = 0

    # Safely fetch values with column names
    # Austin TX is at approx [30.75, -97.48]
    try:
        lat = float(dff.loc[row, LAT_COL])
        lon = float(dff.loc[row, LON_COL])
        breed = str(dff.loc[row, BREED_COL])
        name = str(dff.loc[row, NAME_COL])
    except KeyError:
        # If any columns are missing, fall back to center-only map
        return [
            dl.Map(
                style={"width": "1000px", "height": "500px"},
                center=[30.75, -97.48],
                zoom=10,
                children=[dl.TileLayer(id="base-layer-id")],
            )
        ]

    return [
        dl.Map(
            style={"width": "1000px", "height": "500px"},
            center=[lat, lon],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(breed),
                        dl.Popup(
                            [
                                html.H1("Animal Name"),
                                html.P(name),
                            ]
                        ),
                    ],
                ),
            ],
        )
    ]


# Run app and display result in jupyterlab mode
# Note: if the default port (8050) is busy, you can pass port=8051 (or another) to run_server.
app.run_server()


Dash app running on https://tahitigloria-galaxygizmo-3000.codio.io/proxy/8050/
